# Stream Model From GCP Cloud Storage

In this notebook we will demonstrate how to read tensors using the RunAI Model Streamer from GCP Cloud Storage bucket.

## Prerequisites
* A Linux machine
* A GCP Cloud Storage bucket
* HMAC Credentials for a GCP Service Account that can read and write to the bucket (roles/storage.objectUser)

Install python libraries

In [ ]:
!pip install boto3 runai-model-streamer runai-model-streamer-s3

## Preparation
Set the following variables with your credentials and bucket:

In [1]:
gcs_bucket = 'your-bucket-name'
gcs_location = 'desired/path/in/gcs'

# See https://cloud.google.com/storage/docs/authentication/hmackeys
# Configured through https://console.cloud.google.com/storage/settings/interoperability
gcs_hmac_access_key = 'your-access-key'
gcs_hmac_access_secret = 'your-secret-access-key'

Now we will set them as Cloud Storage environment varibles:

In [2]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = gcs_hmac_access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = gcs_hmac_access_secret

gcs_endpoint = 'https://storage.googleapis.com'
os.environ['AWS_ENDPOINT_URL'] = gcs_endpoint

# As of boto3 1.36.0, new integrity protections for checksum calculation are
# applied by default. Disable these for now, to maintain compatibility.
# See https://github.com/boto/boto3/issues/4392
os.environ['AWS_REQUEST_CHECKSUM_CALCULATION'] = 'when_required'

We will start by downloading an example `.safetensors` file. Feel free to use your own.

Next thing we would do is upload the `model.safetensors` file to the Cloud Storage bucket:

In [4]:
import boto3

local_file = 'model.safetensors'
gcs_upload_location = f"{gcs_location}/{local_file}"

gcs_client = boto3.client(
    's3',
    region_name='auto',
    aws_access_key_id=gcs_hmac_access_key,
    aws_secret_access_key=gcs_hmac_access_secret,
    endpoint_url=gcs_endpoint
)

gcs_client.upload_file(local_file, gcs_bucket, gcs_upload_location)

## Streaming

To load the tensors from the Cloud Storage file we need to create `SafetensorsStreamer` instance, perform the request, and start getting the tensors:

In [ ]:
from runai_model_streamer import SafetensorsStreamer

# Replace "gs://" with "s3://"
file_path = f"s3://{gcs_bucket}/{gcs_upload_location}"

def heavy_workload(tensor):
    # Perform heavy computation with the tensor
    # The computation is occured during the reading
    # of the rest of the tensors from the storage
    return

with SafetensorsStreamer() as streamer:
    streamer.stream_file(file_path)
    for name, tensor in streamer.get_tensors():
        heavy_workload(tensor)